In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
auliayasmin_music_mood_classification_path = kagglehub.dataset_download('auliayasmin/music-mood-classification')

print('Data source import complete.')


100%|██████████| 1.95G/1.95G [00:23<00:00, 89.6MB/s]

Extracting files...


Data source import complete.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import librosa
import librosa.display

In [3]:
# To clean up the notebook
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [4]:
# Extracting data into dataframe
rootPath = "/kaggle/input/music-mood-classification/dataset"
paths = []
moods = []
for mood in os.listdir(rootPath):
    for file in os.listdir(rootPath + "/" + mood):
        paths.append(rootPath + "/" + mood + "/" + file)
        moods.append(mood)

data = pd.DataFrame(columns=["filePath", "mood"])
data["filePath"] = paths
data["mood"] = moods

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/music-mood-classification/dataset'

In [ ]:
data.head()

# Exploring Data

In [ ]:
# Checking our input data
data["mood"].value_counts()

In [ ]:
samples = []
sr = 22050
for mood in os.listdir(rootPath):
    sample = { "mood" : mood }
    moodData = data[data.mood == mood]

    print(mood + ":")
    IPython.display.display(IPython.display.Audio(moodData.iloc[20, 0]))
    signal, _ = librosa.load(moodData.iloc[30, 0], sr=sr)
    sample["signal"] = signal
    samples.append(sample)

## Plotting Amplitude Time Graphs

In [ ]:
fig = plt.figure(figsize=(20,10))
for i, sample in enumerate(samples):
    plt.subplot(2, 3, i + 1)
    librosa.display.waveshow(sample["signal"], sr=sr, alpha=0.4)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.title(sample["mood"])

plt.show()

## Plotting Mel Spectograms

In [ ]:
fig = plt.figure(figsize=(10,8))

for i, sample in enumerate(samples):
    sample["frequencies"] = librosa.stft(sample["signal"])
    sample["frequencies_db"] = librosa.amplitude_to_db(np.abs(sample["frequencies"]), ref=np.max)
    plt.figure(figsize=(20, 10))
    librosa.display.specshow(sample["frequencies_db"])
    plt.xlabel("Time")
    plt.ylabel("Amplitude (db)")
    plt.title(sample["mood"])
    plt.colorbar()

plt.show()

## Chromagrams

In [ ]:
fig = plt.figure(figsize=(10,8))

for i, sample in enumerate(samples):
    sample["chroma"] = librosa.feature.chroma_stft(y=sample["signal"], sr=sr)
    plt.figure(figsize=(20, 10))
    librosa.display.specshow(sample["chroma"], y_axis='chroma', x_axis='time')
    plt.title(sample["mood"])
    plt.colorbar()

plt.show()

## Visuaizing MFCCs

In [ ]:
fig = plt.figure(figsize=(10,8))

for i, sample in enumerate(samples):
    sample["mfccs"] = librosa.feature.mfcc(y=sample["signal"], sr=sr, n_mfcc=40)
    plt.figure(figsize=(20, 10))
    librosa.display.specshow(sample["mfccs"], x_axis='time')
    plt.title(sample["mood"])
    plt.colorbar()

plt.show()

## Feature Extraction

The model will be trained with the tempo and the average values of the mel spectrogram, chroma_stft, and mfccs.

In [ ]:
def getFeatures(filePath):
    signal, sr = librosa.load(filePath)

    tempo = librosa.beat.tempo(y=signal)
    output = np.array(tempo)

    stft = np.mean(librosa.feature.melspectrogram(y=signal).T, axis=0)
    output = np.hstack((output, stft))

    chroma = np.mean(librosa.feature.chroma_stft(y=signal).T, axis=0)
    output = np.hstack((output, chroma))

    mfccs = np.mean(librosa.feature.mfcc(y=signal).T, axis=0)
    output = np.hstack((output, mfccs))

    return output

print(getFeatures(data.iloc[0, 0]))

In [ ]:
print("Extracting Features...")
X, y = [], []

for i, row in data.iterrows():
    label = row["mood"]
    features = getFeatures(row["filePath"])
    X.append(features)
    y.append(label)
    if i % 100 == 0:
        print("#", end="")
print("\ndone")

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X

In [ ]:
y

# Splitting Data into Test and Train Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# encoding string labels ("happy", "sad", ...) to OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_train_ohe = ohe.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_ohe = ohe.fit_transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
y_train_oe = oe.fit_transform(y_train.reshape(-1, 1))
y_test_oe = oe.fit_transform(y_test.reshape(-1, 1))

In [ ]:
y_train_oe

In [ ]:
y_train_ohe

# Defining Evaluation Method for Models

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.metrics as skm

def modelResults(y_train, y_pred):
    ConfusionMatrixDisplay.from_predictions(y_train, y_pred)
    ConfusionMatrixDisplay.from_predictions(y_train, y_pred, normalize="true", values_format=".0%")
    print(skm.classification_report(y_train, y_pred, digits=3))
    print(skm.precision_recall_fscore_support(y_train, y_pred, average="macro"))

# Trying Logistic Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict


scaler = StandardScaler()
lr = LogisticRegression(max_iter=400)
lr_pipeline = make_pipeline(scaler, lr)

y_train_pred = cross_val_predict(lr_pipeline, X_train, y_train_oe, cv=3)

y_train_pred

In [ ]:
modelResults(y_train_oe, y_train_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomForestClassifier = RandomForestClassifier()
randomForestClassifier.fit(X_train, y_train_oe)
y_train_pred = cross_val_predict(randomForestClassifier, X_train, y_train_oe, cv=3)

y_train_pred

modelResults(y_train_oe, y_train_pred)

# Final Evaluation Using Random Forest

In [ ]:
model = randomForestClassifier
y_test_pred = model.predict(X_test)
modelResults(y_test_oe, y_test_pred)

# Testing on my favorite songs
Say you won't let go - James Arthur

Personally, I would classify this song as Romantic, but if it ends up being sad then I'll take it

In [ ]:
say_you_wont_let_go_path = "/kaggle/input/personal-favorite-songs/Say You Wont Let Go.mp3"
IPython.display.display(IPython.display.Audio(say_you_wont_let_go_path))

In [ ]:
def get5sFeatures(filePath):
    signal, sr = librosa.load(filePath)

    tempo = librosa.beat.tempo(y=signal)
    output = np.array(tempo)

    signal = signal[sr*20:sr * 25]
    print(signal.shape)

    stft = np.mean(librosa.feature.melspectrogram(y=signal).T, axis=0)
    output = np.hstack((output, stft))

    chroma = np.mean(librosa.feature.chroma_stft(y=signal).T, axis=0)
    output = np.hstack((output, chroma))

    mfccs = np.mean(librosa.feature.mfcc(y=signal).T, axis=0)
    output = np.hstack((output, mfccs))

    return output

x = get5sFeatures(say_you_wont_let_go_path)

In [ ]:
pred = model.predict(x.reshape(1, -1))

In [ ]:
pred_transform = oe.inverse_transform(pred.reshape(1, -1))
pred_transform